# 🚀​ Preparación de conjuntos de datos para consultas
En este notebook se preparan datos necesarios para hacer las consultas en la API.

#### 📥Importaciones 

In [1]:
from textblob import TextBlob
import pandas as pd
import json
import ast 
import re
import pyarrow as pa
import pyarrow.parquet as pq 

#### 📦 Extracción de los datos

Se extrean los datos de `df_juegos` que contiene la información relacionada a los juegos que tiene la plataforma.

In [2]:
df_juegos = pd.read_csv('Data/steam_games_limpio.csv', encoding='utf-8')

Se extrean los datos de `df_items` que contiene la información relacionada al consumo de los juegos por parte de los usuarios.

In [3]:
df_items = pd.read_csv('Data/users_items_limpio.csv', encoding='utf-8')

Se extrean los datos de `df_reviews` que contiene la información relacionada a los usuarios que realizaron review de los juegos que consumen.

In [4]:
df_reviews = pd.read_csv('Data/user_review_limpio.csv', encoding='utf-8')

### 👍​​👎​ Análisis de sentimientos

Se pide crear una nueva columna llamada 'sentiment_analysis' que reemplace a 'reviews_review' donde se realice un análisis de sentimiento de los comentarios con la siguiente escala:

* 0 si es malo.
* 1 si es neutral.
* 2 si es positivo.

Dado que el objetivo de este proyecto es realizar una prueba de concepto, consiguiendo un producto mínimo viable, se realiza un análisis de sentimiento básico utilizando ``TextBlob`` que es una biblioteca de procesamiento de lenguaje natural (NLP) en Python. El objetivo de esta metodología es asignar un valor numérico a un texto, en este caso a los comentarios que los usuarios dejaron para un juego determinado, para representar si el sentimiento expresado en el texto es negativo, neutral o positivo. 

Esta metodología toma una revisión de texto como entrada, utiliza ``TextBlob`` para calcular la polaridad de sentimiento y luego clasifica la revisión como negativa, neutral o positiva en función de la polaridad calculada. En este caso, se consideraron las polaridades por defecto del modelo, el cuál utiliza umbrales -0.2 y 0.2, siendo polaridades negativas por debajo de -0.2, positivas por encima de 0.2 y neutrales entre medio de ambos.

Primero se crea una funcion para que realice el análisis de sentimiento en un texto dado y devuelva un valor numérico que represente el sentimiento.

In [5]:
def analisis_sentimiento(review):
    if review is None:
        return 1
    analysis = TextBlob(review)
    polarity = analysis.sentiment.polarity
    if polarity < -0.2:
        return 0  
    elif polarity > 0.2: 
        return 2 
    else:
        return 1 

In [6]:
df_reviews['sentiment_analysis'] = df_reviews['reviews_review'].apply(analisis_sentimiento)
df_reviews.head()

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_review,reviews_date,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,2011-11-05,1
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,2014-06-24,1
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,A suitably punishing roguelike platformer. Wi...,Formato inválido,2
3,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,This game... is so fun. The fight sequences ha...,2013-10-14,2
4,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,Git gud,2014-04-15,1


Se crea una función que imprime ejemplos de reviews para cada categoría de análisis de sentimiento.

In [7]:
def ejemplos_review_por_sentimiento(reviews, sentiments):
    for sentiment_value in range(3):
        print(f"Para la categoría de análisis de sentimiento {sentiment_value} se tienen estos ejemplos de reviews:")
        sentiment_reviews = [review for review, sentiment in zip(reviews, sentiments) if sentiment == sentiment_value]
        
        for i, review in enumerate(sentiment_reviews[:3], start=1):
            print(f"Review {i}: {review}")
        
        print("\n")

Se revisan algunos ejemplos para cada una de las clases de sentimiento.

In [8]:
ejemplos_review_por_sentimiento(df_reviews['reviews_review'], df_reviews['sentiment_analysis'])

Para la categoría de análisis de sentimiento 0 se tienen estos ejemplos de reviews:
Review 1: This game is Marvellous.
Review 2: Killed the Emperor, nobody cared and got away with it. Accidentally killed a chicken and everybody decided to gang up on me. 10/10
Review 3: This Game Doesn't Work


Para la categoría de análisis de sentimiento 1 se tienen estos ejemplos de reviews:
Review 1: Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.
Review 2: I know what you think when you see this title "Barbie Dreamhouse Party" but do not be intimidated by it's title, this is easily one of my GOTYs. You don't get any of that cliche game mechanics that all the latest games have, this is simply good core gameplay. Yes, you can't 360 noscope your friends, but what you can do is show them up with your bad 

Se elimina la columna de 'reviews_review'.

In [9]:
df_reviews = df_reviews.drop(columns=['reviews_review'])
df_reviews.columns

Index(['user_id', 'user_url', 'reviews_item_id', 'reviews_helpful',
       'reviews_recommend', 'reviews_date', 'sentiment_analysis'],
      dtype='object')

Se revisan los tipos de datos por columna y la cantidad de nulos.

In [10]:
def verificar_tipo_datos(df):
    
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}

    for columna in df.columns:
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100-porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())

    df_info = pd.DataFrame(mi_dict)
            
    return df_info

In [11]:
verificar_tipo_datos(df_reviews)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,user_id,[<class 'str'>],100.0,0.0,0
1,user_url,[<class 'str'>],100.0,0.0,0
2,reviews_item_id,[<class 'int'>],100.0,0.0,0
3,reviews_helpful,[<class 'str'>],100.0,0.0,0
4,reviews_recommend,[<class 'bool'>],100.0,0.0,0
5,reviews_date,[<class 'str'>],100.0,0.0,0
6,sentiment_analysis,[<class 'int'>],100.0,0.0,0


#### 💱 Transformacion de `df_reviews`: agregamos la columna 'release_anio'

Se agrega el titulo, el año de lanzamiento de un juego y la empresa desarrolldora del mismo al dataframe `df_reviews`. En primer lugar se extraen las columnas de id del juego, el titulo del juego y el año de lanzamiento del mismo, se borran los duplicados y luego se unen con el dataframe de reviews.

In [12]:
anio_lanzamiento_item = df_juegos[['id', 'title', 'release_anio', 'developer']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
anio_lanzamiento_item = anio_lanzamiento_item.rename(columns={'id':'reviews_item_id'})
# se eliminan los duplicados
anio_lanzamiento_item = anio_lanzamiento_item.drop_duplicates()
anio_lanzamiento_item

,reviews_item_id,title,release_anio,developer
0,761140,Lost Summoner Kitty,2018,Kotoshiro
5,643980,Ironbound,2018,Secret Level SRL
9,670290,Real Pool 3D - Poolians,2017,Poolians.com
14,767400,弹炸人2222,2017,彼岸领域
17,772540,Battle Royale Trainer,2018,Trickjump Games Ltd
...,...,...,...,...
71535,745400,Kebab it Up!,2018,Bidoniera Games
71539,773640,Colony On Mars,2018,"Nikita ""Ghost_RUS"""
71543,733530,LOGistICAL: South Africa,2018,Sacada
71546,610660,Russian Roads,2018,Laush Dmitriy Sergeevich


In [13]:
df_reviews = df_reviews.merge(anio_lanzamiento_item, on='reviews_item_id')
df_reviews

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,title,release_anio,developer
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011-11-05,1,Killing Floor,2009,Tripwire Interactive
1,EndAtHallow,http://steamcommunity.com/id/EndAtHallow,1250,No ratings yet,True,2015-01-15,1,Killing Floor,2009,Tripwire Interactive
2,76561198077432581,http://steamcommunity.com/profiles/76561198077...,1250,No ratings yet,True,2014-12-12,1,Killing Floor,2009,Tripwire Interactive
3,76561198057958244,http://steamcommunity.com/profiles/76561198057...,1250,0 of 1 people (0%) found this review helpful,True,2013-12-13,0,Killing Floor,2009,Tripwire Interactive
4,46366536564574576346346546,http://steamcommunity.com/id/46366536564574576...,1250,2 of 3 people (67%) found this review helpful,True,2014-08-19,1,Killing Floor,2009,Tripwire Interactive
...,...,...,...,...,...,...,...,...,...,...
48797,ButtBurger2,http://steamcommunity.com/id/ButtBurger2,73010,No ratings yet,True,2012-05-17,0,Cities in Motion,2011,Colossal Order Ltd.
48798,76561198064526566,http://steamcommunity.com/profiles/76561198064...,378930,3 of 17 people (18%) found this review helpful,False,Formato inválido,1,Pesadelo - Regressão,2016,Skyjaz Games
48799,haungaraho,http://steamcommunity.com/id/haungaraho,16600,No ratings yet,True,2012-10-22,2,Trials 2: Second Edition,2008,Redlynx
48800,UnseenPrecision,http://steamcommunity.com/id/UnseenPrecision,232950,No ratings yet,True,2014-01-19,1,Bridge Project,2013,Halycon Media GmbH &amp; Co. KG


Se elimina la columna 'user_url'.

In [14]:
df_reviews.drop('user_url', axis=1, inplace=True)
df_reviews

,user_id,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,title,release_anio,developer
0,76561197970982479,1250,No ratings yet,True,2011-11-05,1,Killing Floor,2009,Tripwire Interactive
1,EndAtHallow,1250,No ratings yet,True,2015-01-15,1,Killing Floor,2009,Tripwire Interactive
2,76561198077432581,1250,No ratings yet,True,2014-12-12,1,Killing Floor,2009,Tripwire Interactive
3,76561198057958244,1250,0 of 1 people (0%) found this review helpful,True,2013-12-13,0,Killing Floor,2009,Tripwire Interactive
4,46366536564574576346346546,1250,2 of 3 people (67%) found this review helpful,True,2014-08-19,1,Killing Floor,2009,Tripwire Interactive
...,...,...,...,...,...,...,...,...,...
48797,ButtBurger2,73010,No ratings yet,True,2012-05-17,0,Cities in Motion,2011,Colossal Order Ltd.
48798,76561198064526566,378930,3 of 17 people (18%) found this review helpful,False,Formato inválido,1,Pesadelo - Regressão,2016,Skyjaz Games
48799,haungaraho,16600,No ratings yet,True,2012-10-22,2,Trials 2: Second Edition,2008,Redlynx
48800,UnseenPrecision,232950,No ratings yet,True,2014-01-19,1,Bridge Project,2013,Halycon Media GmbH &amp; Co. KG


## 📥 Carga de los dataframe

In [16]:
dfs = [df_reviews, df_juegos, df_items]
# Nombres correspondientes a cada DataFrame
names = ['df_reviews', 'df_games', 'df_items']

for df, name in zip(dfs, names):
    archivo = f'data/{name}_unido.csv'
    df.to_csv(archivo, index=False, encoding='utf-8')
    print(f"DataFrame '{name}' guardado como '{archivo}'")

DataFrame 'df_reviews' guardado como 'data/df_reviews_unido.csv'
DataFrame 'df_games' guardado como 'data/df_games_unido.csv'
DataFrame 'df_items' guardado como 'data/df_items_unido.csv'


Para optimizar la estructura de los datos en el deploy, se aprovecha en este punto a guardar los dataframe en formato parquet.

In [17]:
for df, name in zip(dfs, names):
    archivo = f'data/parquet/{name}.parquet'
    pq.write_table(pa.Table.from_pandas(df), archivo)
    print(f"DataFrame '{name}' guardado como '{archivo}'")

DataFrame 'df_reviews' guardado como 'data/parquet/df_reviews.parquet'
DataFrame 'df_games' guardado como 'data/parquet/df_games.parquet'
DataFrame 'df_items' guardado como 'data/parquet/df_items.parquet'
